# VWAP (Volume Weighted Average Price)
**O que é:**  
O VWAP é o preço médio ponderado pelo volume negociado ao longo do dia, indicando o preço “justo” intradiário do ativo.  
**Por que usar:**  
Ajuda a avaliar se o preço atual está alto ou baixo em relação ao volume negociado. É útil para traders institucionais e intradiários, pois mostra onde a maior parte do volume foi negociada.  
**Boas práticas:**  
- Uso intradiário, reiniciado a cada sessão (normalmente um dia).  
- Preço acima do VWAP pode indicar tendência de alta ou ativo “caro” intradiário. Abaixo, tendência de baixa ou “barato”.  
- Combine com análise de price action e outros indicadores para confirmar entradas e saídas.  

In [ ]:
from dotenv import load_dotenv
import os
import pandas as pd
import ccxt

load_dotenv()

binance_api_key = os.getenv("BINANCE_API_KEY")
binance_api_secret = os.getenv("BINANCE_API_SECRET")

binance = ccxt.binance({
    "enableRateLimit": True,
    "options": {
        "defaultType": "future"
    },
    "apiKey": binance_api_key,
    "secret": binance_api_secret,
})

symbol = "BTCUSDT"
timeframe = "1h"
bars = binance.fetch_ohlcv(symbol, timeframe, limit=500)

df_candles = pd.DataFrame(bars, columns=["time", "abertura", "max", "min", "fechamento", "volume"])
df_candles["time"] = pd.to_datetime(df_candles["time"], unit="ms", utc=True).map(lambda x: x.tz_convert("America/Sao_Paulo"))

In [4]:
df_candles["preco_ponderado"] = df_candles["fechamento"] * df_candles["volume"]
df_candles["VWAP"] = df_candles["preco_ponderado"].sum() / df_candles["volume"].sum()

In [6]:
df_candles

,time,abertura,max,min,fechamento,volume,preco_ponderado,VWAP
0,2024-11-28 01:00:00-03:00,95523.1,95884.5,95394.6,95796.1,5361.889,5.136481e+08,99646.834535
1,2024-11-28 02:00:00-03:00,95796.2,95985.1,95410.2,95513.5,4730.965,4.518710e+08,99646.834535
2,2024-11-28 03:00:00-03:00,95513.5,95771.0,95135.8,95544.9,6717.816,6.418531e+08,99646.834535
3,2024-11-28 04:00:00-03:00,95544.8,95830.6,95270.0,95371.6,4228.817,4.033090e+08,99646.834535
4,2024-11-28 05:00:00-03:00,95371.5,95690.3,95141.8,95522.9,5310.152,5.072411e+08,99646.834535
...,...,...,...,...,...,...,...,...
495,2024-12-18 16:00:00-03:00,104771.7,104794.4,102777.1,102782.1,31059.597,3.192371e+09,99646.834535
496,2024-12-18 17:00:00-03:00,102781.8,102954.8,100301.5,100836.1,75472.021,7.610304e+09,99646.834535
497,2024-12-18 18:00:00-03:00,100839.3,101726.5,100457.2,100982.3,16691.613,1.685557e+09,99646.834535
498,2024-12-18 19:00:00-03:00,100982.2,101528.0,100620.1,101122.4,9316.223,9.420788e+08,99646.834535
